## 1. 공시 사업 보고서 로드 및 전처리

해당 실습은 형태소 분석기 Mecab의 원활한 설치를 위해서 구글의 Colab에서 진행했다고 가정합니다. 여러분들의 컴퓨터에 Mecab을 설치하였거나, 다른 형태소 분석기를 사용한다면 Colab에서 하지 않더라도 상관없습니다.

In [ ]:
# dart.csv 파일 다운로드
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1nIUZxASrbIa0Z2xzGVgX1dqjtnFCoKNT' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1nIUZxASrbIa0Z2xzGVgX1dqjtnFCoKNT" -O dart.csv && rm -rf /tmp/cookies.txt

In [ ]:
# 형태소 분석기 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm

In [ ]:
df = pd.read_csv('/content/dart.csv',  sep=',')
df = df.dropna()
df

In [ ]:
mecab = Mecab()

tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
  text = row['business']
  tag = row['name']
  tagged_corpus_list.append(TaggedDocument(tags=[tag], words=mecab.morphs(text)))

print('문서의 수 :', len(tagged_corpus_list))

In [ ]:
tagged_corpus_list[0]

## 2. Doc2Vec 학습 및 테스트
이제 모델을 학습합니다. 사업 보고서가 꽤 긴 문서인데다 토큰화 외에 불용어 제거 등 별도 추가 전처리를 진행하지 않았기 때문에 저자가 Colab에서 수행했을 때, 소요 시간이 최소 1시간 이상 걸렸습니다.


In [ ]:
from gensim.models import doc2vec

In [ ]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('dart.doc2vec')

In [ ]:
similar_doc = model.docvecs.most_similar('동화약품')
print(similar_doc)

In [ ]:
similar_doc = model.docvecs.most_similar('하이트진로')
print(similar_doc)

In [ ]:
similar_doc = model.docvecs.most_similar('LG이노텍')
print(similar_doc)